In [1]:
# office_activation_demo.py
# Simple, legal simulation of an Office activation flow for demo/teaching purposes.
# Requirements: pip install flask

from flask import Flask, request, render_template_string, redirect, url_for, flash
import re
import uuid
from datetime import datetime

app = Flask(__name__)
app.secret_key = "dev-demo-secret"  # for flashing messages (don't use in production)

# Mock "database" of product keys for demo only (DOES NOT ACTIVATE anything)
# Put some sample keys here to demonstrate success vs failure.
MOCK_KEYS = {
    "AAAAA-BBBBB-CCCCC-DDDDD-EEEEE": {"product": "Office 2021 Pro", "type": "retail"},
    "11111-22222-33333-44444-55555": {"product": "Office 2021 Std", "type": "volume"},
}

# HTML templates (inline for single-file demo)
INDEX_HTML = """
<!doctype html>
<title>Office Activation Demo</title>
<h1>Office Activation - Demo (Simulation)</h1>
<p>This is a <strong>simulated</strong> activation flow for demonstration. No real activation occurs.</p>
<form action="{{ url_for('activate') }}" method="post">
  <label for="key">Enter Product Key (format: XXXXX-XXXXX-XXXXX-XXXXX-XXXXX)</label><br>
  <input id="key" name="key" autofocus required style="width: 380px; padding:6px" placeholder="AAAAA-BBBBB-CCCCC-DDDDD-EEEEE"><br><br>
  <button type="submit">Activate</button>
</form>
{% with messages = get_flashed_messages(with_categories=true) %}
  {% if messages %}
    <ul style="list-style:none; padding:0; margin-top:16px;">
    {% for category, msg in messages %}
      <li style="padding:8px; border-radius:6px; background: {% if category=='success' %}#e6ffe6{% elif category=='error' %}#ffe6e6{% else %}#f0f0f0{% endif %}; margin-bottom:8px;">{{ msg|safe }}</li>
    {% endfor %}
    </ul>
  {% endif %}
{% endwith %}
<hr>
<p>For your project: show the activation request, demonstrate validation, and explain how a real activation would contact Microsoft's servers and exchange activation tokens. This demo only simulates that exchange.</p>
"""

RESULT_HTML = """
<!doctype html>
<title>Activation Result</title>
<h1>Activation Result (Simulated)</h1>
{% if success %}
  <p><strong>Product:</strong> {{ product }}</p>
  <p><strong>Activation ID:</strong> <code>{{ activation_id }}</code></p>
  <p><strong>Licensing Type:</strong> {{ lic_type }}</p>
  <p style="color:green;"><strong>Status:</strong> Activated (simulated)</p>
{% else %}
  <p style="color:red;"><strong>Activation failed:</strong> {{ message }}</p>
  <p><a href="{{ url_for('index') }}">Try another key</a></p>
{% endif %}
<hr>
<p><a href="{{ url_for('index') }}">Back to demo</a></p>
"""

# Simple regex to validate a 5x5 product key format (letters/numbers)
KEY_REGEX = re.compile(r'^[A-Z0-9]{5}(?:-[A-Z0-9]{5}){4}$', re.IGNORECASE)


def log_attempt(key, success, info=""):
    """Append a human-readable log entry to demo_activation.log."""
    stamp = datetime.utcnow().isoformat() + "Z"
    with open("demo_activation.log", "a", encoding="utf-8") as f:
        f.write(f"{stamp}\t{'SUCCESS' if success else 'FAIL'}\t{key}\t{info}\n")


@app.route("/", methods=["GET"])
def index():
    return render_template_string(INDEX_HTML)


@app.route("/activate", methods=["POST"])
def activate():
    key = request.form.get("key", "").strip().upper()
    # 1) Validate format
    if not KEY_REGEX.match(key):
        flash("Invalid product key format. Expect 5 groups of 5 letters/numbers (e.g. AAAAA-BBBBB-CCCCC-DDDDD-EEEEE).", "error")
        log_attempt(key, False, "invalid-format")
        return redirect(url_for("index"))

    # 2) Simulate server-side check against local mock DB
    record = MOCK_KEYS.get(key)
    if not record:
        # Simulate possible server responses: key not found / key blocked
        flash("Product key not found in demo database. (This demo does not contact Microsoft.)", "error")
        log_attempt(key, False, "key-not-found")
        return redirect(url_for("index"))

    # 3) On "success", generate a fake activation id (UUID)
    activation_id = str(uuid.uuid4()).upper()
    log_attempt(key, True, f"activated-as-{record['product']} id={activation_id}")
    # Show a result page
    return render_template_string(RESULT_HTML, success=True, product=record["product"], activation_id=activation_id, lic_type=record.get("type", "unknown"))


if __name__ == "__main__":
    app.run(debug=True, port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\VIRAT RAI\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
